In [1]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat

urls = ["https://www.youtube.com/watch?v=8JQl8FXUZ8c",
       "https://www.youtube.com/watch?v=J7KEJzPksnc",
       "https://www.youtube.com/watch?v=P2P7Dcbp4wI"]

In [2]:
import os
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

model_name = os.getenv("OPENAI_MODEL_NAME")

class SumarizeChain():
    def __init__(self, chain_type="refine"):
        self.llm = ChatOpenAI(temperature=0, model_name=model_name)
        self.chain_type = chain_type

        question_prompt_template = """
                        Forneça um resumo em tópicos dos assuntos tratados no texto a seguir.
                        TEXTO: {text}
                        RESUMO:
                        """

        question_prompt = PromptTemplate(
            template=question_prompt_template, input_variables=["text"]
        )

        refine_prompt_template = """
                    Escreva um resumo conciso do texto a seguir delimitado por crases triplas.
                    Retorne sua resposta em tópicos que cubram todos os pontos-chave do texto.
                    ```{text}```
                    RESUMO DOS PONTOS:
                    """

        refine_prompt = PromptTemplate(
            template=refine_prompt_template, input_variables=["text"]
        )

        if chain_type == "refine":
            self.refine_chain = load_summarize_chain(
                self.llm,
                chain_type="refine",
                question_prompt=question_prompt,
                refine_prompt=refine_prompt,
                return_intermediate_steps=True,
            )
        elif chain_type == "stuff":
            self.refine_chain = load_summarize_chain(
                self.llm,
                chain_type="stuff",
                prompt=question_prompt,
            )

    def summarize(self, document):
        refine_outputs = self.refine_chain({"input_documents": document})
        return refine_outputs["output_text"]

    def summarize_text(self, text):
        document = [Document(page_content=text, metadata={"source": "context"})]
        return self.summarize(document)


class SumarizeVideo():
    name: str = "Resumir Vídeo"
    description: str = (
        "Resume a transcrição de um vídeo do YouTube."
    )

    def _run(self, url_video: str) -> str:
        llm = ChatOpenAI(temperature=0, model_name=model_name)

        loader = YoutubeLoader.from_youtube_url(
            url_video,
            add_video_info=True,
            language=["pt","en"],
        )
        document = loader.load()

        summarize_chain = SumarizeChain()
        refine_outputs = summarize_chain.summarize(document)

        return refine_outputs

In [8]:
summarize_chain = SumarizeVideo()
refine_outputs = summarize_chain._run(urls[2])

In [9]:
from IPython.display import display, Markdown
Markdown(refine_outputs)

**Resumo em Tópicos: Dispensacionalismo e suas Vertentes**

1. **Introdução ao Tema:**
   - Vídeo sobre dispensacionalismo apresentado pelo pastor Iago Martins.
   - Uso de camiseta de Lutero como provocação.

2. **Objetivo do Vídeo:**
   - Continuar a série sobre dispensacionalismo.
   - Explicar o dispensacionalismo revisado e progressivo.

3. **Histórico do Dispensacionalismo:**
   - Vídeos anteriores abordaram a história e doutrinas fundamentais do dispensacionalismo clássico.
   - Afirmação de que o dispensacionalismo clássico não é mais seguido.

4. **Dispensacionalismo Revisado:**
   - Abandono do dualismo entre humanidade celestial e terrena.
   - Distinção entre Israel e a Igreja, com diferentes prerrogativas e responsabilidades.

5. **Natureza da Salvação:**
   - Salvação é a mesma para ambos os grupos (Israel e Igreja).
   - Distinção eterna entre Israel e a Igreja.

6. **Concepções de Vida Eterna:**
   - Dispensacionalistas revisados divergem sobre a natureza da vida eterna.
   - Algumas visões incluem todos os remidos no céu ou na Nova Terra.

7. **Estrutura das Dispensações:**
   - Manutenção da divisão das dispensações clássicas.
   - Propósitos de Deus variam entre Israel e a Igreja em diferentes dispensações.

8. **Mudanças na Interpretação Bíblica:**
   - Abandono gradual da tipologia em favor de uma interpretação literal.
   - Influência de movimentos hermenêuticos e a complexidade da hermenêutica atual.

9. **Alianças Bíblicas:**
   - Aceitação da aliança abraâmica como fundamental.
   - A Igreja vista como a semente espiritual de Abraão.

10. **Reino de Deus vs. Reino dos Céus:**
    - Distinção entre os dois reinos foi criticada e atualmente não é mais aceita.
    - Várias visões alternativas do Reino foram propostas.

11. **Conclusão:**
    - Revisões ao dispensacionalismo clássico se tornaram amplamente aceitas.
    - O dispensacionalismo revisado prepara o caminho para o dispensacionalismo progressivo, que será abordado em vídeos futuros.

12. **Interação com o Público:**
    - Convite para comentários e discussões sobre as diferentes perspectivas do dispensacionalismo.
    - Incentivo à inscrição no canal para mais conteúdos teológicos.

In [ ]:
class SearchTranscript():
    name: str = "Busca em Transcrição"
    description: str = (
        "Busca o tópico nos vídeos do YouTube e retorna os dados."
    )

    def _run(self, url_video: str, human_input: str) -> str:
        app = App.from_config(config_path="youtube_video.yaml")
        app.add(url_video, data_type="youtube_video")
        results = app.search(f"O que é dito no vídeo sobre: {human_input}", where={"url": url_video})

        output = ""
        output += f"### Tópico: {human_input}\n\n"

        for i in range(len(results)):
            transcript = ast.literal_eval(results[i]['metadata']['transcript'])
            context = results[i]['context']
            #output += f"**Contexto:** {context}\n\n"

            output += f"**Resumo:** {SumarizeChain(chain_type="stuff").summarize_text(text=context)}\n\n"

            # Inicializar variáveis para encontrar o chunk com menor start time
            min_start_time = None
            selected_chunk = None

            # Agrupar chunks de 5 em 5 com 2 sobrepostos
            grouped_chunks = []
            for j in range(0, len(transcript), 2):  # Passo de 3 para sobrepor 2 chunks
                group = transcript[j:j + 5]
                if group:  # Garantir que o grupo não está vazio
                    grouped_chunks.append(group)

            for group in grouped_chunks:
                # Unir os textos dos chunks agrupados
                combined_text = ' '.join(chunk['text'] for chunk in group)

                # Verifica se o texto combinado aparece no contexto
                if combined_text in context:
                    for chunk in group:
                        if min_start_time is None or chunk['start'] < min_start_time:
                            min_start_time = chunk['start']
                            selected_chunk = chunk

            # Se encontrou um chunk correspondente, gera o link
            if selected_chunk:
                segundos = f"{str(selected_chunk['start'])}s"
                title = results[i]['metadata']['title'] + " - " + segundos
                output += f"**Abra o vídeo neste ponto:** [{title}]({url_video}&t={segundos})\n\n"

        return output